# Random Forest

## Data Preparation

In [ ]:
# Import Necessary Packages and Mount the Drive
!pip install scikit-learn --upgrade
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold, RandomizedSearchCV
pd.set_option('display.max_columns', None)
from preprocessing.preprocessing import one_hot_encode_days, one_hot_encode_months, time_differencing, split_and_scale, sin_cos_waves, sliding_windows

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

%cd '/content/gdrive/MyDrive/ECSE_552/Project'

In [ ]:
# Import the dataset
df = pd.read_csv('data/ON_demand_weather_17-20.csv', index_col=0)

# Do all the preprocessing required
# Remove weather stations and market demand columns
df.drop(['Market Demand',
         'toronto_Temp (C)',
         'hamilton_Temp (C)',
         'ottawa_Temp (C)',
         'kitchener_Temp (C)',
         'london_Temp (C)',
         'windsor_Temp (C)'], axis=1, inplace=True)

# Apply scaling if desired and split dataset into train, validation, and testset
scaler = None  # Define the type of scaler, options are 'min-max', 'standard', and None
columns_to_scale = ['Ontario Demand', 'Weighted Average Temp (C)']  # Define columns to be scaled, add time-differenced column if applicable
target_column = columns_to_scale[0]  # Define target column (required to allow unscaling later), change to time-differenced column if applicable
vali_set_start_date='2019-07-01'  # First day of validation set
test_set_start_date='2020-01-01'  # First day of test set

train_df, vali_df, test_df, target_scaler = split_and_scale(df,
                                                            scaler=scaler,
                                                            columns_to_scale=columns_to_scale,
                                                            target_column=target_column,
                                                            vali_set_start_date=vali_set_start_date,
                                                            test_set_start_date=test_set_start_date)

## Model Training and Evaluation

#### 1Hour Prediction

In [ ]:
# Prepare Sliding Window for 1H Prediction
window_size = 18
flatten = True
output_window_size = 1
perform_feature_shift = False

# Sliding windows
# note that the function removes the 'Date' column (not numeric)
x_train, y_train = sliding_windows(df=train_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_vali, y_vali = sliding_windows(df=vali_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_test, y_test = sliding_windows(df=test_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)

In [ ]:
# Train random forest
model = RandomForestRegressor(n_estimators=100)
# model = RandomForestRegressor(n_estimators=100, min_samples_split=3, min_samples_leaf=2, max_depth=50)

model.fit(x_train, y_train)

# R2 score
print("RF train R^2-Score: %0.3f" % model.score(x_train, y_train))
print("RF valid. R^2-Score: %0.3f" % model.score(x_vali, y_vali))

# RMSE (scaled, if applicable)
y_train_pred = model.predict(x_train)
y_vali_pred = model.predict(x_vali)

print("RF train RMSE: %0.3f" % sqrt(mean_squared_error(y_train_pred, y_train)))
print("RF valid. RMSE: %0.3f" % sqrt(mean_squared_error(y_vali_pred, y_vali)))

if scaler is not None:
  y_train_unscaled = target_scaler.inverse_transform(y_train)
  y_vali_unscaled = target_scaler.inverse_transform(y_vali)
  y_train_pred_unscaled = target_scaler.inverse_transform(y_train_pred.reshape(-1, 1))
  y_vali_pred_unscaled = target_scaler.inverse_transform(y_vali_pred.reshape(-1, 1))
  print("RF train RMSE (unscaled): %0.3f" % sqrt(mean_squared_error(y_train_pred_unscaled, y_train_unscaled)))
  print("RF valid. RMSE (unscaled): %0.3f" % sqrt(mean_squared_error(y_vali_pred_unscaled, y_vali_unscaled)))


In [ ]:
# Plot Performance
samples = [4,81,854,1701,3621]
for i in samples:
  plt.figure(figsize=(10,2))
  plt.plot(y_vali_pred[i], label='pred')
  plt.plot(y_vali[i], label='true')
  plt.legend()
  plt.xlabel('hours')
  plt.ylabel('Demand in MW')
  plt.title('Random Forest prediction example on validation set')
  plt.show()

##### Testing

In [ ]:
for i in range(1,6):
    print('Test-Run ',i)
    print('------------------------------')
    print('------------------------------')
    model = RandomForestRegressor(n_estimators=100)
    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_vali_pred = model.predict(x_vali)
    y_test_pred = model.predict(x_test)

    print("RF train RMSE: %0.3f" % sqrt(mean_squared_error(y_train, y_train_pred)))
    print("RF valid. RMSE: %0.3f" % sqrt(mean_squared_error(y_vali, y_vali_pred)))
    print("RF test. RMSE: %0.3f" % sqrt(mean_squared_error(y_test, y_test_pred)))
    print("RF test. MAPE: %0.4f" % (mean_absolute_percentage_error(y_test, y_test_pred)))

    print('------------------------------')
    print('------------------------------')

#### 24 Hour Sequence Prediction

In [ ]:
# Get sliding window for 24 hour prediction
window_size = 48
flatten = True
output_window_size = 24
perform_feature_shift = False

# Sliding windows
# note that the function removes the 'Date' column (not numeric)
x_train, y_train = sliding_windows(df=train_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_vali, y_vali = sliding_windows(df=vali_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_test, y_test = sliding_windows(df=test_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)

# Adjust target vectors, keep only last element
y_train = y_train[:, -1]
y_vali = y_vali[:, -1]

In [ ]:
# Train random forest
model = RandomForestRegressor(n_estimators=500)
# model = RandomForestRegressor(n_estimators=100, min_samples_split=3, min_samples_leaf=2, max_depth=50)

model.fit(x_train, y_train)

# R2 score
print("RF train R^2-Score: %0.3f" % model.score(x_train, y_train))
print("RF valid. R^2-Score: %0.3f" % model.score(x_vali, y_vali))

# RMSE (scaled, if applicable)
y_train_pred = model.predict(x_train)
y_vali_pred = model.predict(x_vali)

print("RF train RMSE: %0.3f" % sqrt(mean_squared_error(y_train_pred, y_train)))
print("RF valid. RMSE: %0.3f" % sqrt(mean_squared_error(y_vali_pred, y_vali)))

if scaler is not None:
  y_train_unscaled = target_scaler.inverse_transform(y_train)
  y_vali_unscaled = target_scaler.inverse_transform(y_vali)
  y_train_pred_unscaled = target_scaler.inverse_transform(y_train_pred.reshape(-1, 1))
  y_vali_pred_unscaled = target_scaler.inverse_transform(y_vali_pred.reshape(-1, 1))
  print("RF train RMSE (unscaled): %0.3f" % sqrt(mean_squared_error(y_train_pred_unscaled, y_train_unscaled)))
  print("RF valid. RMSE (unscaled): %0.3f" % sqrt(mean_squared_error(y_vali_pred_unscaled, y_vali_unscaled)))

In [ ]:
# Plot Performance
samples = [4,81,854,1701,3621]
for i in samples:
  plt.figure(figsize=(10,2))
  plt.plot(y_vali_pred[i], label='pred')
  plt.plot(y_vali[i], label='true')
  plt.legend()
  plt.xlabel('hours')
  plt.ylabel('Demand in MW')
  plt.title('Random Forest prediction example on validation set')
  plt.show()

##### Testing

In [ ]:
for i in range(1,6):
    print('Test-Run ',i)
    print('------------------------------')
    print('------------------------------')
    model = RandomForestRegressor(n_estimators=100)
    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_vali_pred = model.predict(x_vali)
    y_test_pred = model.predict(x_test)

    print("RF train RMSE: %0.3f" % sqrt(mean_squared_error(y_train, y_train_pred)))
    print("RF valid. RMSE: %0.3f" % sqrt(mean_squared_error(y_vali, y_vali_pred)))
    print("RF test. RMSE: %0.3f" % sqrt(mean_squared_error(y_test, y_test_pred)))
    print("RF test. MAPE: %0.4f" % (mean_absolute_percentage_error(y_test, y_test_pred)))

    print('------------------------------')
    print('------------------------------')

#### 24Hour AutoRegressive
The 24Hour AutoRegressive Model for random forest is based on the 1H model.

In [ ]:
# Prepare Sliding Window for 1H Prediction
window_size = 18
flatten = True
output_window_size = 1
perform_feature_shift = False

# Sliding windows
# note that the function removes the 'Date' column (not numeric)
x_train, y_train = sliding_windows(df=train_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_vali, y_vali = sliding_windows(df=vali_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_test, y_test = sliding_windows(df=test_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)

# Adjust target vectors, keep only last element
y_train = y_train[:, -1]
y_vali = y_vali[:, -1]

In [ ]:
# Train random forest
model = RandomForestRegressor(n_estimators=100)
# model = RandomForestRegressor(n_estimators=100, min_samples_split=3, min_samples_leaf=2, max_depth=50)

model.fit(x_train, y_train)

# R2 score
print("RF train R^2-Score: %0.3f" % model.score(x_train, y_train))
print("RF valid. R^2-Score: %0.3f" % model.score(x_vali, y_vali))

# RMSE (scaled, if applicable)
y_train_pred = model.predict(x_train)
y_vali_pred = model.predict(x_vali)

print("RF train RMSE: %0.3f" % sqrt(mean_squared_error(y_train_pred, y_train)))
print("RF valid. RMSE: %0.3f" % sqrt(mean_squared_error(y_vali_pred, y_vali)))

if scaler is not None:
  y_train_unscaled = target_scaler.inverse_transform(y_train)
  y_vali_unscaled = target_scaler.inverse_transform(y_vali)
  y_train_pred_unscaled = target_scaler.inverse_transform(y_train_pred.reshape(-1, 1))
  y_vali_pred_unscaled = target_scaler.inverse_transform(y_vali_pred.reshape(-1, 1))

  # for i in range(len(y_train_pred_unscaled)):
    # print(y_vali_unscaled[i], y_vali_pred_unscaled[i])

  # print(y_train_unscaled.shape)
  # print(y_vali_unscaled.shape)
  # print(y_train_pred_unscaled.shape)
  # print(y_vali_pred_unscaled.shape)

  print("RF train RMSE (unscaled): %0.3f" % sqrt(mean_squared_error(y_train_pred_unscaled, y_train_unscaled)))
  print("RF valid. RMSE (unscaled): %0.3f" % sqrt(mean_squared_error(y_vali_pred_unscaled, y_vali_unscaled)))


In [ ]:
# Autoregressive predictions
pred_length = 24  # number of steps done per prediction
dataset = vali_df.copy()  # define dataset (before sliding window)
dataset.drop(columns='Date', inplace=True)
dataset[target_column] = dataset.pop(target_column)
dataset = dataset.to_numpy()

# Create dataframe to store losses
loss_df = pd.DataFrame(columns=[j for j in range(1, pred_length+1)])
# loss_unscaled_df = pd.DataFrame(columns=[j for j in range(1, pred_length+1)])

true_values = []

for i in range(dataset.shape[0] - window_size - pred_length+1):
  # Get first query for each sequence
  query = dataset[i:i+window_size].reshape(dataset.shape[1]*window_size)
  # print(query)
  for j in range(pred_length):
    # Predict
    pred = model.predict(query.reshape(1,-1))

    # Extract true value + additional data from next row
    next_row = dataset[i+j+window_size:i+j+window_size+1]
    true_value = next_row[:,-1]
    additional_data = next_row[:,:-1][0]

    # Compute loss and add to dataframes
    loss = mean_squared_error(pred, true_value)
    # loss_unscaled = mean_squared_error(scaler.inverse_transform(pred), scaler.inverse_transform(true_value))
    loss_df.loc[i, j+1] = loss
    # loss_unscaled_df.loc[i, j+1] = loss_unscaled

    # Get true value of last time step
    if j == (pred_length-1):
      true_values.append(true_value)

    # Create new query window from data + prediction
    new_row = np.concatenate((additional_data, pred),axis=None)
    query = np.concatenate((query[dataset.shape[1]:], new_row), axis=None)

In [ ]:
# GET SQUAREROOT OF VALUES
# CURRENTLY REPORTING MSE BELOW

# Plot results
plt.title('Random Forest autoregressive error')
plt.ylabel('MSE')
plt.xlabel('Forecasted time-steps')
plt.plot(loss_df.mean())
plt.show()

##### Testing

In [ ]:
for i in range(1,6):
    print('Test-Run ',i)
    print('------------------------------')
    print('------------------------------')
    model = RandomForestRegressor(n_estimators=100)
    model.fit(x_train, y_train)

    y_train_pred = model.predict(x_train)
    y_vali_pred = model.predict(x_vali)
    y_test_pred = model.predict(x_test)

    print("RF train RMSE: %0.3f" % sqrt(mean_squared_error(y_train, y_train_pred)))
    print("RF valid. RMSE: %0.3f" % sqrt(mean_squared_error(y_vali, y_vali_pred)))
    print("RF test. RMSE: %0.3f" % sqrt(mean_squared_error(y_test, y_test_pred)))
    print("RF test. MAPE: %0.4f" % (mean_absolute_percentage_error(y_test, y_test_pred)))



    # Create dataframe to store losses
    loss_df = pd.DataFrame(columns=[j for j in range(1, pred_length + 1)])
    loss_mape_df = pd.DataFrame(columns=[j for j in range(1, pred_length+1)])

    true_values = []

    for i in range(dataset.shape[0] - window_size - pred_length + 1):
        # Get first query for each sequence
        query = dataset[i:i + window_size].reshape(dataset.shape[1] * window_size)
        # print(query)
        for j in range(pred_length):
            # Predict
            pred = model.predict(query.reshape(1, -1))

            # Extract true value + additional data from next row
            next_row = dataset[i + j + window_size:i + j + window_size + 1]
            true_value = next_row[:, -1]
            additional_data = next_row[:, :-1][0]

            # Compute loss and add to dataframes
            loss = mean_squared_error(true_value, pred)
            loss_mape = mean_absolute_percentage_error(true_value, pred)
            # loss_unscaled = mean_squared_error(scaler.inverse_transform(pred), scaler.inverse_transform(true_value))
            loss_df.loc[i, j + 1] = loss
            loss_mape_df.loc[i, j + 1] = loss_mape
            # loss_unscaled_df.loc[i, j+1] = loss_unscaled

            # Get true value of last time step
            if j == (pred_length - 1):
                true_values.append(true_value)

            # Create new query window from data + prediction
            new_row = np.concatenate((additional_data, pred), axis=None)
            query = np.concatenate((query[dataset.shape[1]:], new_row), axis=None)

    print('Sequence RMSE: {:.3f}'.format(sqrt(loss_df.mean().mean())))
    print('Sequence MAPE: {:.4f}'.format(loss_mape_df.mean().mean()))

    print('------------------------------')
    print('------------------------------')